In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import epanet as epa
import epamodule as em
import seaborn as sns
from cjsbot import CjsBot
from scipy import optimize
from scipy import stats
import time
import os 
from multiprocessing import Process, Lock

## Determina $T_0$

Initial temperature $T_0$: it may be calculated as a preliminary step using the following algorithm:

- initiate 100 disturbances at random; evaluate the average $\left<\Delta E\right>$ of the corresponding $\Delta E$ variations;
- choose an initial rate of acceptance tetha of the degrading perturbations according to the assumed quality of the initial configuration; for example:
    - "poor" quality: tetha = 50%(starting at high temperature)
    - "good" quality: tetha = 20% (low temp.)
- deduce $T_0$ from the relation $\exp(-\left<\Delta E\right>/T_0)$

In [2]:
def determina_acceptance(x0, f, T, n=100):
    pontos = np.random.random((len(x), n))*1 +0.001
    energias = []
    for i in range(1,n):
        energias.append(f(pontos[:,i]) - f(pontos[:,i-1]))
    media = np.array(energias).mean()
    return np.exp(-media/T)

In [3]:
def determina_t0(x0, f, n=100, tetha=0.2):
    pontos = np.random.random((len(x), n))*1 +0.001
    energias = []
    for i in range(1,n):
        energias.append(f(pontos[:,i]) - f(pontos[:,i-1]))
    media = np.array(energias).mean()
    return -media/np.log(tetha)

## On the acceleration of simulated annealing
James D. Cohoow

In [4]:
def simulated_annealing(x0, f, min_score=0.1, t0=100, t_min = 0.1, alpha=0.9, iter_MAX=50, perturb=0.01):
    x = x0
    x_bsf = x0
    t = t0 #aquece até a temperatura t0
    imagem = []
    caminho = []
    caminho2 = []
    caminho.append(x0)
    caminho2.append(x0)
    metropolis = []
    validation = []
    while t > t_min and f(x_bsf)>min_score: # até que atinja a temperatura mínima para parar
        #if len(validation) > 10:
        #    if validation[-1] == validation[-10]:
        #        return x_bsf, imagem, caminho, caminho2, metropolis
                #return simulated_annealing(x_bsf, f, min_score=min_score, t0=t0, dt=dt, iter_MAX=iter_MAX)
        for i in range(iter_MAX):# até que atinja o equilíbrio na temperatura atual
            if f(x_bsf) < 1:
                y = np.round(x.copy(), decimals=3)
                r = np.random.randint(0,len(x))
                y[r] += np.random.choice([-0.001,0.001])
            else:
                y = x + np.random.normal(0,perturb,len(x)) # perturbe o x 
            delta_xy = f(y) - f(x) # faça a variação do custo, se negativa, então y tem um custo menor
            metropolis.append(np.exp(-delta_xy/t))
            if (delta_xy <= 0) or np.random.uniform(0,1) < np.exp(-delta_xy/t): # a aceitação do novo ponto segue o critério de metropolis
                x = y
                caminho2.append(x)
                if f(x) < f(x_bsf): # caso o ponto seja aceito pelo critério de metropolis, ele é avaliado
                    x_bsf = x 
                    caminho.append(x_bsf)
            imagem.append(f(x_bsf))
        validation.append(f(x_bsf))
        print(f"t={t}\tf(x)={f(x_bsf)}")
        t = t*alpha
    return x_bsf, imagem, caminho, caminho2, metropolis

In [17]:
t = np.array([0.075, 0.812, 0.317, 0.581, 0.752, 0.994, 0.967, 0.511, 0.851,
              0.925, 0.842, 0.295, 0.633, 0.522, 0.306])
seeds = [661, 308, 769, 343, 491]
posicao_nos = [0.1, 0.3, 0.5, 0.7, 0.9]
q_nos = [10, 20, 30, 40, 50]   
dim=14
seed = seeds[0]
posicao_no = posicao_nos[2]
q_no = q_nos[3]
links = ["../../networks/c-town/nodes", 
        "../../networks/c-town/links", 
        "../../networks/c-town/rede.inp", 
        f"../../networks/c-town/{dim}dim_{posicao_no}_{q_no}.csv"]

diretorio = f'./teste/'
if not os.path.isdir(diretorio):
    os.mkdir(diretorio)
rv = epa.RealValuesNos(links, t[:dim], nos_dim=q_no, posicao=posicao_no)
rv.getRealValue()
rv.close_sim()
net = epa.Rede(links, rv.groups, t[:dim], [0.001,1])

Começando simulação


In [6]:
x0 = np.random.random((dim,))*1+0.001
x, imagem, caminho, caminho2, metropolis = simulated_annealing(x0, net.objetivo, min_score=1e-10, 
                                                   t_min=1e-20, t0=10, alpha=0.9 , iter_MAX=100)

t=10	f(x)=201.73075434973927
t=9.0	f(x)=201.73075434973927
t=8.1	f(x)=188.65655695885943
t=7.29	f(x)=118.58100484736593
t=6.561	f(x)=88.90543598149733
t=5.9049000000000005	f(x)=80.79085131209114
t=5.3144100000000005	f(x)=60.516673990560896
t=4.7829690000000005	f(x)=37.030404479467585
t=4.3046721	f(x)=29.727116713772443
t=3.8742048900000006	f(x)=29.727116713772443
t=3.4867844010000004	f(x)=29.727116713772443
t=3.1381059609000004	f(x)=29.727116713772443
t=2.82429536481	f(x)=29.727116713772443
t=2.541865828329	f(x)=29.727116713772443
t=2.2876792454961	f(x)=29.727116713772443
t=2.05891132094649	f(x)=29.4491021314098
t=1.853020188851841	f(x)=22.02609046994627
t=1.6677181699666568	f(x)=13.651971040097163
t=1.5009463529699911	f(x)=11.046549254226202
t=1.350851717672992	f(x)=9.286445159646084
t=1.2157665459056928	f(x)=6.3118989990425876
t=1.0941898913151236	f(x)=6.3118989990425876
t=0.9847709021836112	f(x)=6.3118989990425876
t=0.88629381196525	f(x)=6.011978667192137
t=0.7976644307687251	f(x)=6

<ipython-input-4-097730bbc8ac>:25: RuntimeWarning: overflow encountered in exp
  metropolis.append(np.exp(-delta_xy/t))


t=3.13163795527598e-07	f(x)=0.040179316782085764
t=2.818474159748382e-07	f(x)=0.038728223065844496
t=2.5366267437735436e-07	f(x)=0.03753725431215463
t=2.2829640693961893e-07	f(x)=0.036820283319683664
t=2.0546676624565705e-07	f(x)=0.036041560395459346
t=1.8492008962109135e-07	f(x)=0.035270822585515504
t=1.6642808065898222e-07	f(x)=0.03404658013005117
t=1.49785272593084e-07	f(x)=0.03189457444568551
t=1.348067453337756e-07	f(x)=0.030459556292952526
t=1.2132607080039804e-07	f(x)=0.029045253658294842
t=1.0919346372035823e-07	f(x)=0.028199268074785122
t=9.82741173483224e-08	f(x)=0.027613462489169818
t=8.844670561349016e-08	f(x)=0.026951909179841697
t=7.960203505214115e-08	f(x)=0.02632382217689025
t=7.164183154692704e-08	f(x)=0.025777008459043498
t=6.447764839223434e-08	f(x)=0.02453227963087425
t=5.802988355301091e-08	f(x)=0.023151887693528765
t=5.2226895197709816e-08	f(x)=0.02235025730988018
t=4.700420567793884e-08	f(x)=0.021793642389637925
t=4.2303785110144955e-08	f(x)=0.021295936856403127


t=1.0896840314361235e-14	f(x)=0.011864202845571162
t=9.807156282925112e-15	f(x)=0.011864202845571162
t=8.826440654632602e-15	f(x)=0.011864202845571162
t=7.943796589169342e-15	f(x)=0.011864202845571162
t=7.149416930252408e-15	f(x)=0.011864202845571162
t=6.434475237227167e-15	f(x)=0.011864202845571162
t=5.79102771350445e-15	f(x)=0.011864202845571162
t=5.211924942154005e-15	f(x)=0.011864202845571162
t=4.690732447938605e-15	f(x)=0.011864202845571162
t=4.2216592031447445e-15	f(x)=0.011864202845571162
t=3.79949328283027e-15	f(x)=0.011864202845571162
t=3.4195439545472436e-15	f(x)=0.011864202845571162
t=3.0775895590925193e-15	f(x)=0.011864202845571162
t=2.7698306031832673e-15	f(x)=0.011864202845571162
t=2.4928475428649408e-15	f(x)=0.011864202845571162
t=2.2435627885784466e-15	f(x)=0.011864202845571162
t=2.019206509720602e-15	f(x)=0.011864202845571162
t=1.817285858748542e-15	f(x)=0.011864202845571162
t=1.6355572728736877e-15	f(x)=0.011864202845571162
t=1.472001545586319e-15	f(x)=0.0118642028455

In [7]:
net.get_dist(x)

0.13159787232322565

In [18]:
x0 = np.random.random((dim,))*1+0.001
x, imagem, caminho, caminho2, metropolis = simulated_annealing(x0, net.objetivo, min_score=1e-10, 
                                                   t_min=1e-20, t0=10, alpha=0.9 , iter_MAX=100)

t=10	f(x)=103.12053755420855
t=9.0	f(x)=97.52663513995593
t=8.1	f(x)=97.52663513995593
t=7.29	f(x)=97.52663513995593
t=6.561	f(x)=67.1737211105692
t=5.9049000000000005	f(x)=64.22062268448559
t=5.3144100000000005	f(x)=64.22062268448559
t=4.7829690000000005	f(x)=64.22062268448559
t=4.3046721	f(x)=64.22062268448559
t=3.8742048900000006	f(x)=64.22062268448559
t=3.4867844010000004	f(x)=64.22062268448559
t=3.1381059609000004	f(x)=55.99274465688902
t=2.82429536481	f(x)=55.99274465688902
t=2.541865828329	f(x)=44.7369803011743
t=2.2876792454961	f(x)=30.48712068429721
t=2.05891132094649	f(x)=24.845227055775467
t=1.853020188851841	f(x)=24.845227055775467
t=1.6677181699666568	f(x)=24.845227055775467
t=1.5009463529699911	f(x)=24.845227055775467
t=1.350851717672992	f(x)=24.845227055775467
t=1.2157665459056928	f(x)=24.845227055775467
t=1.0941898913151236	f(x)=24.845227055775467
t=0.9847709021836112	f(x)=24.845227055775467
t=0.88629381196525	f(x)=24.845227055775467
t=0.7976644307687251	f(x)=24.8452270

t=7.960203505214115e-08	f(x)=24.845227055775467
t=7.164183154692704e-08	f(x)=24.845227055775467
t=6.447764839223434e-08	f(x)=24.845227055775467
t=5.802988355301091e-08	f(x)=24.845227055775467
t=5.2226895197709816e-08	f(x)=24.845227055775467
t=4.700420567793884e-08	f(x)=24.845227055775467
t=4.2303785110144955e-08	f(x)=24.845227055775467
t=3.807340659913046e-08	f(x)=24.845227055775467
t=3.426606593921742e-08	f(x)=24.845227055775467
t=3.0839459345295674e-08	f(x)=24.845227055775467
t=2.7755513410766107e-08	f(x)=24.845227055775467
t=2.4979962069689497e-08	f(x)=24.845227055775467
t=2.2481965862720548e-08	f(x)=24.845227055775467
t=2.0233769276448492e-08	f(x)=24.845227055775467
t=1.8210392348803643e-08	f(x)=24.845227055775467
t=1.638935311392328e-08	f(x)=24.845227055775467
t=1.4750417802530952e-08	f(x)=24.845227055775467
t=1.3275376022277857e-08	f(x)=24.845227055775467
t=1.1947838420050071e-08	f(x)=24.845227055775467
t=1.0753054578045064e-08	f(x)=24.845227055775467
t=9.677749120240559e-09	f(x)

t=1.3248013910276873e-15	f(x)=24.845227055775467
t=1.1923212519249186e-15	f(x)=24.845227055775467
t=1.0730891267324267e-15	f(x)=24.845227055775467
t=9.65780214059184e-16	f(x)=24.845227055775467
t=8.692021926532656e-16	f(x)=24.845227055775467
t=7.822819733879391e-16	f(x)=24.845227055775467
t=7.0405377604914515e-16	f(x)=24.845227055775467
t=6.336483984442307e-16	f(x)=24.845227055775467
t=5.702835585998076e-16	f(x)=24.845227055775467
t=5.132552027398269e-16	f(x)=24.845227055775467
t=4.619296824658442e-16	f(x)=24.845227055775467
t=4.1573671421925985e-16	f(x)=24.845227055775467
t=3.741630427973339e-16	f(x)=24.845227055775467
t=3.3674673851760054e-16	f(x)=24.845227055775467
t=3.030720646658405e-16	f(x)=24.845227055775467
t=2.7276485819925645e-16	f(x)=24.845227055775467
t=2.4548837237933083e-16	f(x)=24.845227055775467
t=2.2093953514139775e-16	f(x)=24.845227055775467
t=1.9884558162725798e-16	f(x)=24.845227055775467
t=1.7896102346453218e-16	f(x)=24.845227055775467
t=1.6106492111807896e-16	f(x)=

In [19]:
x-net.target

array([-0.04217819, -0.30023652,  0.2515401 , -0.03420391, -0.32429327,
       -0.55041287, -0.36152588, -0.2252545 ,  0.06658752, -0.22485138,
       -0.1027077 ,  0.08895705,  0.33089232,  0.37194765])

In [12]:
net.get_dist(x)

0.1221187946222857

In [13]:
t = np.array([0.075, 0.812, 0.317, 0.581, 0.752, 0.994, 0.967, 0.511, 0.851,
              0.925, 0.842, 0.295, 0.633, 0.522, 0.306])
seeds = [661, 308, 769, 343, 491]
posicao_nos = [0.1, 0.3, 0.5, 0.7, 0.9]
q_nos = [10, 20, 30, 40, 50]   
dim=3
seed = seeds[0]
posicao_no = posicao_nos[2]
q_no = q_nos[3]
links = ["../../networks/c-town/nodes", 
        "../../networks/c-town/links", 
        "../../networks/c-town/rede.inp", 
        f"../../networks/c-town/{dim}dim_{posicao_no}_{q_no}.csv"]

diretorio = f'./teste/'
if not os.path.isdir(diretorio):
    os.mkdir(diretorio)
rv = epa.RealValuesNos(links, t[:dim], nos_dim=q_no, posicao=posicao_no)
rv.getRealValue()
rv.close_sim()
net = epa.Rede(links, rv.groups, t[:dim], [0.001,1])
x0 = np.random.random((dim,))*1+0.001
x, imagem, caminho, caminho2, metropolis = simulated_annealing(x0, net.objetivo, min_score=1e-10, 
                                                   t_min=1e-20, t0=10, alpha=0.9 , iter_MAX=100)

Começando simulação
t=10	f(x)=86.29736000065724
t=9.0	f(x)=38.161916877512375
t=8.1	f(x)=38.161916877512375
t=7.29	f(x)=38.161916877512375
t=6.561	f(x)=38.161916877512375
t=5.9049000000000005	f(x)=18.94893037961814
t=5.3144100000000005	f(x)=18.94893037961814
t=4.7829690000000005	f(x)=0.949504991202059
t=4.3046721	f(x)=0.7508014530776657
t=3.8742048900000006	f(x)=0.7139575476821941
t=3.4867844010000004	f(x)=0.7094512485273391
t=3.1381059609000004	f(x)=0.7094512485273391
t=2.82429536481	f(x)=0.7094512485273391
t=2.541865828329	f(x)=0.7094512485273391
t=2.2876792454961	f(x)=0.7094512485273391
t=2.05891132094649	f(x)=0.7094512485273391
t=1.853020188851841	f(x)=0.7094512485273391
t=1.6677181699666568	f(x)=0.7094512485273391
t=1.5009463529699911	f(x)=0.6494875657408634
t=1.350851717672992	f(x)=0.5752652360461142
t=1.2157665459056928	f(x)=0.5752652360461142
t=1.0941898913151236	f(x)=0.5752652360461142
t=0.9847709021836112	f(x)=0.5752652360461142
t=0.88629381196525	f(x)=0.5752652360461142
t=0.

In [14]:
net.get_dist(x)

1.1102230246251565e-16

In [15]:
t = np.array([0.075, 0.812, 0.317, 0.581, 0.752, 0.994, 0.967, 0.511, 0.851,
              0.925, 0.842, 0.295, 0.633, 0.522, 0.306])
seeds = [661, 308, 769, 343, 491]
posicao_nos = [0.1, 0.3, 0.5, 0.7, 0.9]
q_nos = [10, 20, 30, 40, 50]   
dim=10
seed = seeds[0]
posicao_no = posicao_nos[2]
q_no = q_nos[3]
links = ["../../networks/c-town/nodes", 
        "../../networks/c-town/links", 
        "../../networks/c-town/rede.inp", 
        f"../../networks/c-town/{dim}dim_{posicao_no}_{q_no}.csv"]

diretorio = f'./teste/'
if not os.path.isdir(diretorio):
    os.mkdir(diretorio)
rv = epa.RealValuesNos(links, t[:dim], nos_dim=q_no, posicao=posicao_no)
rv.getRealValue()
rv.close_sim()
net = epa.Rede(links, rv.groups, t[:dim], [0.001,1])
x0 = np.random.random((dim,))*1+0.001
x, imagem, caminho, caminho2, metropolis = simulated_annealing(x0, net.objetivo, min_score=1e-10, 
                                                   t_min=1e-20, t0=10, alpha=0.9 , iter_MAX=100)

Começando simulação
t=10	f(x)=69.54101623679617
t=9.0	f(x)=51.435042066082985
t=8.1	f(x)=50.042255919683136
t=7.29	f(x)=31.73920685098979
t=6.561	f(x)=31.73920685098979
t=5.9049000000000005	f(x)=27.77860363401209
t=5.3144100000000005	f(x)=27.77860363401209
t=4.7829690000000005	f(x)=27.77860363401209
t=4.3046721	f(x)=27.77860363401209
t=3.8742048900000006	f(x)=20.971267875586765
t=3.4867844010000004	f(x)=20.272828623368667
t=3.1381059609000004	f(x)=20.272828623368667
t=2.82429536481	f(x)=13.265158614445655
t=2.541865828329	f(x)=10.029161050899331
t=2.2876792454961	f(x)=7.425186100498636
t=2.05891132094649	f(x)=7.425186100498636
t=1.853020188851841	f(x)=7.425186100498636
t=1.6677181699666568	f(x)=7.425186100498636
t=1.5009463529699911	f(x)=7.425186100498636
t=1.350851717672992	f(x)=7.100275652231185
t=1.2157665459056928	f(x)=7.100275652231185
t=1.0941898913151236	f(x)=6.483150042083243
t=0.9847709021836112	f(x)=6.2066948662331916
t=0.88629381196525	f(x)=5.984371409875153
t=0.797664430768

t=1.2132607080039804e-07	f(x)=0.0006804867207390198
t=1.0919346372035823e-07	f(x)=0.0006804867207390198
t=9.82741173483224e-08	f(x)=0.0006804867207390198
t=8.844670561349016e-08	f(x)=0.0006804867207390198
t=7.960203505214115e-08	f(x)=0.0006804867207390198
t=7.164183154692704e-08	f(x)=0.0006804867207390198
t=6.447764839223434e-08	f(x)=0.0006804867207390198
t=5.802988355301091e-08	f(x)=0.0006804867207390198
t=5.2226895197709816e-08	f(x)=0.0006804867207390198
t=4.700420567793884e-08	f(x)=0.0006804867207390198
t=4.2303785110144955e-08	f(x)=0.0006804867207390198
t=3.807340659913046e-08	f(x)=0.0006804867207390198
t=3.426606593921742e-08	f(x)=0.0006804867207390198
t=3.0839459345295674e-08	f(x)=0.0006804867207390198
t=2.7755513410766107e-08	f(x)=0.0006804867207390198
t=2.4979962069689497e-08	f(x)=0.0006804867207390198
t=2.2481965862720548e-08	f(x)=0.0006804867207390198
t=2.0233769276448492e-08	f(x)=0.0006804867207390198
t=1.8210392348803643e-08	f(x)=0.0006804867207390198
t=1.638935311392328e-0

t=5.79102771350445e-15	f(x)=0.0006804867207390198
t=5.211924942154005e-15	f(x)=0.0006804867207390198
t=4.690732447938605e-15	f(x)=0.0006804867207390198
t=4.2216592031447445e-15	f(x)=0.0006804867207390198
t=3.79949328283027e-15	f(x)=0.0006804867207390198
t=3.4195439545472436e-15	f(x)=0.0006804867207390198
t=3.0775895590925193e-15	f(x)=0.0006804867207390198
t=2.7698306031832673e-15	f(x)=0.0006804867207390198
t=2.4928475428649408e-15	f(x)=0.0006804867207390198
t=2.2435627885784466e-15	f(x)=0.0006804867207390198
t=2.019206509720602e-15	f(x)=0.0006804867207390198
t=1.817285858748542e-15	f(x)=0.0006804867207390198
t=1.6355572728736877e-15	f(x)=0.0006804867207390198
t=1.472001545586319e-15	f(x)=0.0006804867207390198
t=1.3248013910276873e-15	f(x)=0.0006804867207390198
t=1.1923212519249186e-15	f(x)=0.0006804867207390198
t=1.0730891267324267e-15	f(x)=0.0006804867207390198
t=9.65780214059184e-16	f(x)=0.0006804867207390198
t=8.692021926532656e-16	f(x)=0.0006804867207390198
t=7.822819733879391e-16	

In [16]:
net.get_dist(x)

0.006782329983125274